In [1]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

In [2]:
import datasets

dataset=datasets.load_dataset("PiC/phrase_similarity")

/home/ansh/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def similarity_score(word1, word2):
    if word1 not in wv or word2 not in wv:
        return 0
    return wv.similarity(word1, word2)

def sentencesToMatrix(sentence1, sentence2):
    lis1 = sentence1.split()
    lis2 = sentence2.split()
    simMatrix=[]
    for i in lis1:
        if i not in wv:
            continue
        temp=[]
        for j in lis2:
            if j not in wv:
                continue
            temp.append(similarity_score(i,j))
        simMatrix.append(temp)
    return simMatrix

def biggestColumn(matrix):
    maxCol=[]
    for i in matrix:
        i.append(0)
        maxCol.append(max([a for a in i if a<0.95]))
    return maxCol

## BERTSCORE
def getSentenceScore(sentence1, sentence2):
    sentence1=sentence1.lower()
    sentence2=sentence2.lower()
    simMatrix = sentencesToMatrix(sentence1, sentence2)
    maxCol = biggestColumn(simMatrix)
    return sum(maxCol)/len(maxCol)

In [4]:
cur='test'
for i in range(10):
    print(dataset[cur][i]['phrase1'])
    print(dataset[cur][i]['phrase2'])
    print(getSentenceScore(dataset[cur][i]['phrase1'], dataset[cur][i]['phrase2']))
    print(dataset[cur][i]['label'])
    print()

air position
posture while jumping
0.29195863753557205
0

associated track
correlating music single
0.3045092970132828
0

whole parts
extended sections
0.340593621134758
1

wide set
spacious collection
0.17399294674396515
0

full protection
complete defense
0.35668858140707016
0

prior case
preceding game
0.46622487902641296
0

another station
a separate airport
0.2849593162536621
0

initial activity
starting task
0.12714136019349098
1

single square
solitary border
0.21403459459543228
1

independent operation
individual enterprise
0.22330620139837265
0



In [5]:
threshold=0.4
correct=0
cur='test'
for i in range(len(dataset[cur])):
    if getSentenceScore(dataset[cur][i]['phrase1'], dataset[cur][i]['phrase2'])>threshold:
        if dataset[cur][i]['label']==1:
            correct+=1
    else:
        if dataset[cur][i]['label']==0:
            correct+=1
print(correct/len(dataset[cur]))

0.5125
